## Optimizing Resonator Designs

There are many factors for the resonator design. I categorized the factors into two tiers:

Top Tier:
1. Length
2. Center pin to gap ratio (this defines the impedance of the resonator). 

Second Tier:
1. Center pin width


In [2]:
import astropy.units as u
import astropy.constants as c

In [4]:
# define constants for LL designs
sub_t = 350*u.um #substrate thickness (Si in this case)
metal_t = 250*u.nm #Deposited metal thickness (Al)
Sc = 67*u.fF/(u.um)**2 #JJ specific capacitance
epsilon = 11.45
W_jj = 200*u.nm #junction width

### Use Qiskit-metal function to calculate CPW pin/gap ratio

In [8]:
import qiskit_metal.analyses as analyses

In [61]:
freq = 7*u.GHz.si
line_width = 15.322*u.um
line_gap = 10*u.um

In [14]:
freq.to(u.Hz).value

7000000000.0

In [18]:
res = analyses.cpw_calculations.lumped_cpw(freq.to(u.Hz).value,
                                     line_width.to(u.m).value,
                                     line_gap.to(u.m).value,
                                     sub_t.to(u.m).value,
                                     metal_t.to(u.m).value)

Z0 = res[4]*u.Ohm

In [19]:
Z0

<Quantity 52.72897025 Ohm>

In [24]:
def f(line_gap):
    return(analyses.cpw_calculations.lumped_cpw(freq.to(u.Hz).value,
                                     line_width.to(u.m).value,
                                     line_gap.to(u.m).value,
                                     sub_t.to(u.m).value,
                                     metal_t.to(u.m).value)[4]-50)

In [89]:
def find_gap_val( line_gap, tol):
    err = 1
    gap0 = line_gap.value*u.um
    p0 = f(gap0)
    
    step = 0.1*u.um
    gap = line_gap.value*u.um
    pp = f(gap)
    for i in range(1000):
        if (i==0)|(pp*p0>0):
            gap -= step
            pp = f(gap)
        if pp*p0<0:
            # print(pp,p0)
            gap1 = (gap0+gap)/2
            ppp = f(gap1)
            
            if abs(ppp)<tol:
                return(gap1)
            elif ppp*pp<0:
                # print('a1',gap0,gap)
                gap0 = gap1
                gap = gap
                # print('a2',gap0,gap)
            elif p0*ppp<0:
                gap0 = gap0
                gap = gap1
                # print('b',p0,pp,ppp)
                
        # print(p0,pp,ppp)
    
    return(True)

In [90]:
n = find_gap_val(line_gap,0.001)

In [91]:
n

<Quantity 8.28349609 um>

In [87]:
f(n)

0.00034167145241781327

In [ ]:
def find(line_gap,tol):
    err = 1
    